Notebook for processing metadata tables for use in silver. 

NOTE: partially complete b/c accidently lost the notebook

In [3]:
spark.catalog.listTables('test.silver.md') # see metadata tables we made

[Table(name='binary', catalog='test', namespace=['silver', 'md'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='largecat', catalog='test', namespace=['silver', 'md'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='number', catalog='test', namespace=['silver', 'md'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='smallcat', catalog='test', namespace=['silver', 'md'], description=None, tableType='MANAGED', isTemporary=False)]

In [4]:
import numpy as np # for NULL cast since pandas makes NULL nan
sdf = spark.table("test.silver.md.smallcat").toPandas()
unique_fields = np.unique(sdf['field'])
print(sdf, unique_fields)

   silver_column field  code  \
0      DRAT_NULL  DRAT   NaN   
1         DRAT_1  DRAT   1.0   
2         DRAT_2  DRAT   2.0   
3         DRAT_3  DRAT   3.0   
4         DRAT_4  DRAT   4.0   
..           ...   ...   ...   
60        WAOB_4  WAOB   4.0   
61        WAOB_5  WAOB   5.0   
62        WAOB_6  WAOB   6.0   
63        WAOB_7  WAOB   7.0   
64        WAOB_8  WAOB   8.0   

                                                texts  
0   N/A (No service-connected disability/never ser...  
1                                           0 percent  
2                                    10 or 20 percent  
3                                    30 or 40 percent  
4                                    50 or 60 percent  
..                                                ...  
60                  Asia (POBP = 158-159,161,200-299)  
61                Europe (POBP = 100-157,160,162-199)  
62                            Africa (POBP = 400-499)  
63          Northern America (POBP = 300-302,304-309)  

In [9]:
query = "SELECT YEAR, PUMA"
for field in unique_fields:
    if field == 'REGION': # FIXME
        continue
    query += f",\n{field}"
for _, row in sdf.iterrows():
    if row['field'] == 'REGION': # FIXME
        continue
    silver_column, field, code, text = row.values
    match_str = '= ' + str(code) if not np.isnan(code) else 'IS NULL'
    query += f",\nCASE WHEN {field} {match_str} THEN 1 ELSE 0 END {silver_column}"
query += "\nFROM test.bronze.census"
spark.sql(query).writeTo("test.silver.data.smallcat").createOrReplace()

In [10]:
%%sql

SELECT * FROM test.silver.data.smallcat

YEAR,PUMA,DRAT,ENG,MAR,MIL,MSP,PAOC,RAC1P,SCH,WAOB,DRAT_NULL,DRAT_1,DRAT_2,DRAT_3,DRAT_4,DRAT_5,DRAT_6,ENG_NULL,ENG_1,ENG_2,ENG_3,ENG_4,MAR_1,MAR_2,MAR_3,MAR_4,MAR_5,MIL_NULL,MIL_1,MIL_2,MIL_3,MIL_4,MSP_NULL,MSP_1,MSP_2,MSP_3,MSP_4,MSP_5,MSP_6,PAOC_NULL,PAOC_1,PAOC_2,PAOC_3,PAOC_4,RAC1P_1,RAC1P_2,RAC1P_3,RAC1P_4,RAC1P_5,RAC1P_6,RAC1P_7,RAC1P_8,RAC1P_9,SCH_NULL,SCH_1,SCH_2,SCH_3,WAOB_1,WAOB_2,WAOB_3,WAOB_4,WAOB_5,WAOB_6,WAOB_7,WAOB_8
2022,3302,None,None,1,4,2,None,2,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
2022,2900,None,None,5,4,6,None,1,2,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
2022,3100,None,None,1,4,2,None,2,1,1,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
2022,4000,None,None,5,4,6,None,2,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
2022,4300,None,None,3,2,4,None,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
2022,1406,None,None,5,4,6,None,2,3,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
2022,4500,None,None,5,4,6,None,1,3,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
2022,1100,None,None,2,4,3,None,2,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
2022,3000,None,None,2,4,3,None,2,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
2022,1700,None,None,5,4,6,None,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
